<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/notebooks/Station_Buildings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib
!pip install -q osmnx
!pip install -q pydeck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 979.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 761.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.7 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
from tqdm import tqdm
import pydeck as pdk
import shapely
import duckdb
import time

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Deploy Shiny apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


## Setup duckdb

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [4]:
%%sql
INSTALL httpfs;
INSTALL spatial;

,Success


## Download OSM data

In [11]:
## Note: the coordinates are in West, South, East, North format

bboxs = {
    'Mumbai': (72.7712628039,18.8895552464,73.0912396106,19.322318006),
    'Hyderabad': (78.2288393192,17.1952635128,78.692325037,17.6244114807),
    'Bangalore': (77.4480254838,12.815481916,77.7549560258,13.1026335949),
    'Chennai': (80.0337033915,12.8678089117,80.335827415,13.2370438819),
    'Kolkata': (88.2431406976,22.4883995072,88.4944529534,22.7071046611),
    'Ahmedabad': (72.4617185823,22.9179551449,72.699297928,23.1470216925),
    'Delhi_NCR': (76.881197,28.31444,77.604922,28.893765)
}

columns = ['element_type', 'osmid', 'geometry', 'railway', 'internet_access',
       'name', 'name:hi', 'network', 'operator', 'public_transport',
       'ref', 'train','city']

## Select city
city = "Delhi_NCR"
W,S,E,N = bboxs[city]

In [16]:
%%time

public_transport = ox.features.features_from_bbox(
    bbox=(N,S,E,W),
    tags={
        'public_transport':['station', 'stop_position'],
        'railway': 'stop',
        'amenties': 'bus_station',
        'highway': 'bus_stop'
        }
).reset_index()

## Extract the station data:
pb = public_transport[public_transport['element_type']=='node'].copy()
columns = ['element_type', 'osmid', 'geometry', 'railway', 'internet_access',
       'name', 'network', 'operator', 'public_transport',
       'ref', 'train']

pb_gdf = pb[columns]
pb_gdf = pb_gdf.fillna('None')
metro_stations = pb_gdf[pb_gdf['network'].str.lower().str.contains('metro')]
print("Total nodes with station tag:",metro_stations.shape[0])
metro_stations.sample(min(5,len(metro_stations)))

Total nodes with station tag: 763
CPU times: user 1.53 s, sys: 58.7 ms, total: 1.59 s
Wall time: 10.8 s


,element_type,osmid,geometry,railway,internet_access,name,network,operator,public_transport,ref,train
2923,node,5777629582,POINT (77.31142 28.41076),stop,None,Old Faridabad,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,None,None
2489,node,6405278611,POINT (77.38390 28.67850),None,None,Mohan Nagar,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,None,None
61,node,927873513,POINT (77.16172 28.59187),station,None,Dhaula Kuan,Delhi Metro,Delhi Metro Rail Corporation Limited,station,DKV,None
1628,node,5777600349,POINT (77.09266 28.67975),stop,None,Peeragarhi,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,None,None
154,node,3953773598,POINT (77.10513 28.42336),station,None,Sector 55–56,Rapid Metro Gurgaon,Haryana Mass Rapid Transport Corporation Limited,station,None,None


Export the results

In [17]:
metro_stations.to_parquet('stations_stops.parquet')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

In [8]:
pb_gdf.to_file('stations_stops.gpkg', driver='GPKG')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

## Download Station Buildings

Downloading the buildings from overturemaps using duckdb

In [9]:
%%time

buildings_data_url = r"'s3://overturemaps-us-west-2/release/2024-06-13-beta.0/theme=buildings/type=*/*'"
stations_data_url = "'stations_stops.parquet'"

df = duckdb.sql(
    f'''
    LOAD spatial;
    LOAD httpfs;
    COPY(
        SELECT
            id,
            names.primary as name,
            buildings.height as height,
            stations.name as station_name,
            stations.network as network,
            stations.operator as operator,
            stations.railway as railway,
            stations.public_transport as public_transport,
            ST_GeomFromWKB(buildings.geometry) as geom
        FROM read_parquet({buildings_data_url}, filename=true, hive_partitioning=1) AS buildings
        JOIN read_parquet({stations_data_url}) as stations
        ON ST_Intersects(ST_GeomFromWKB(buildings.geometry), ST_GeomFromWKB(stations.geometry))
        WHERE buildings.bbox.xmin > {W}
        AND buildings.bbox.xmax < {E}
        AND buildings.bbox.ymin > {S}
        AND buildings.bbox.ymax < {N}
    ) TO 'station_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
    '''
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 42.9 s, sys: 4.84 s, total: 47.7 s
Wall time: 4min 16s


This will give all the building footprints surrounding the station nodes.
Note that buildings are based on overture maps which is not so well updated. There might me some stations without a enclosing building footprint

Let's find out those stations:

In [10]:
sbf = gpd.read_file('station_buildings.geojson').drop_duplicates('geometry')  ## Unique Building footprints
no_bf = metro_stations[metro_stations[['geometry']].sjoin(sbf, how='left', predicate='intersects').id.isna()].copy()
print('Stations without enclosing buildings:', no_bf.shape[0])
no_bf.sample(5)

Stations without enclosing buildings: 241


/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,element_type,osmid,geometry,railway,internet_access,name,network,operator,public_transport,ref,train
2267,node,4676900919,POINT (77.29579 28.65360),station,None,Karkarduma Court,Delhi Metro,Delhi Metro Rail Corporation Limited,station,None,None
1349,node,2833526614,POINT (77.16543 28.70199),stop,None,Shalimar Bagh,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,SMBG,None
1211,node,268381457,POINT (77.19890 28.66672),station,None,Pratap Nagar,Delhi Metro,Delhi Metro Rail Corporation Limited,station,None,None
1664,node,5777629203,POINT (77.23764 28.65002),stop,None,Jama Masjid,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,JAMD,None
1241,node,554257862,POINT (77.23037 28.65842),stop,None,Chandni Chowk,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,CHK,None


In [11]:
no_bf.to_file('stations_wo_BF.gpkg', driver='GPKG')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Download building footprints from VIDA


[Google-Microsoft Open Buildings - combined by VIDA](https://beta.source.coop/repositories/vida/google-microsoft-open-buildings/description/):

This dataset merges Google's V3 Open Buildings and Microsoft's latest Building Footprints. With 2,534,595,270 footprints, it is to our knowledge the most complete openly available dataset as of Sep. 2023. It covers 92% of Level 0 administrative boundaries, and is divided into 182 partitions. Each footprint is labelled with its respective source, either Google or Microsoft. It can be accessed in cloud-native geospatial formats such as GeoParquet, FlatGeobuf and PMTiles

In [5]:
duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
duckdb.sql('LOAD spatial')

In [6]:
W,S,E,N = 77.3031470176,28.4762710653,77.3070093985,28.4802649353

In [7]:
aoi = shapely.geometry.box(W,S,E,N)


aoi_file = 'aoi.geojson'
gpd.GeoSeries(aoi).to_file(aoi_file)

duckdb.sql(f"CREATE TABLE aoi AS SELECT * FROM ST_Read('{aoi_file}')")
duckdb.sql("SELECT * FROM aoi").show()

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                         geom                                                         │
│                                                       geometry                                                       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((77.3070093985 28.4762710653, 77.3070093985 28.4802649353, 77.3031470176 28.4802649353, 77.3031470176 28.…  │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘



Load the building polygons

In [8]:
%%time
stations_data_url = "'stations_stops.parquet'"
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country"
country_iso = "IND"

duckdb.sql(f'''
describe FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet' AS buildings
''')

CPU times: user 70.1 ms, sys: 24.6 ms, total: 94.6 ms
Wall time: 823 ms


┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ geometry       │ BLOB        │ YES     │ NULL    │ NULL    │ NULL    │
│ boundary_id    │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ bf_source      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ confidence     │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ area_in_meters │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ country_iso    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [13]:


stations_data_url = "'stations_stops.parquet'"
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country"
country_iso = "IND"

query = f'''
SELECT
COUNT (*)
FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet' as buildings
WHERE ST_Xmin(ST_GeomFromWKB(buildings.geometry)) > {W}
AND ST_Xmax(ST_GeomFromWKB(buildings.geometry)) < {E}
AND ST_Ymin(ST_GeomFromWKB(buildings.geometry)) > {S}
AND ST_Ymax(ST_GeomFromWKB(buildings.geometry)) < {N};
'''

count = duckdb.sql(query)

Time taken to process the request: 0.017028 mins


In [14]:
count

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      5419295 │
└──────────────┘

In [18]:
start_time = time.time()

stations_data_url = "'stations_stops.parquet'"
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country"
country_iso = "IND"

query = f'''
COPY(
    SELECT
    ST_Xmin(ST_GeomFromWKB(buildings.geometry)) as xmin,
    ST_Xmax(ST_GeomFromWKB(buildings.geometry)) as xmax,
    ST_Ymin(ST_GeomFromWKB(buildings.geometry)) as ymin,
    ST_Ymax(ST_GeomFromWKB(buildings.geometry)) as ymax,
    confidence,
    bf_source,
    ST_GeomFromWKB(buildings.geometry)
    FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet' as buildings
    JOIN read_parquet({stations_data_url}) as stations
    ON ST_Within(ST_GeomFromWKB(stations.geometry), ST_GeomFromWKB(buildings.geometry))
    WHERE xmin > {W}
    AND xmax < {E}
    AND ymin > {S}
    AND ymax < {N}
) TO 'ML_station_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
'''

duckdb.sql(query)

end_time = (time.time()-start_time)/60

print(f"Time taken to process the request: {end_time:2f} mins")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Time taken to process the request: 16.870243 mins


In [20]:
gdf = gpd.read_file('ML_station_buildings.geojson')
print(gdf.shape)

(519, 7)


In [21]:
gdf.sample(5)

,xmin,xmax,ymin,ymax,confidence,bf_source,geometry
234,77.129482,77.129995,28.636418,28.637716,0.8964,google,"POLYGON ((77.13000 28.63645, 77.12995 28.63671..."
324,77.169043,77.169964,28.588850,28.590031,0.8755,google,"POLYGON ((77.16996 28.58895, 77.16983 28.58914..."
124,77.145782,77.146423,28.672962,28.673130,0.7624,google,"POLYGON ((77.14642 28.67304, 77.14584 28.67310..."
362,77.325957,77.326368,28.570597,28.570958,0.8840,google,"POLYGON ((77.32637 28.57078, 77.32618 28.57096..."
212,77.199047,77.200726,28.643882,28.644744,0.9007,google,"POLYGON ((77.20072 28.64391, 77.20069 28.64392..."


In [37]:
gpd.read_parquet('stations_stops.parquet')

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


,element_type,osmid,geometry,railway,internet_access,name,network,operator,public_transport,ref,train
0,node,249235041,POINT (77.04075 28.59225),stop,None,Dwarka Sector 12,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,DSW,None
1,node,249235042,POINT (77.03356 28.59702),stop,None,Dwarka Sector 13,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,DSTN,None
2,node,249823531,POINT (77.02599 28.60223),stop,None,Dwarka Sector 14,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,DSFN,None
7,node,271335327,POINT (77.07990 28.47952),None,None,M G Road,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,None,None
8,node,271335545,POINT (77.10227 28.48200),None,None,Guru Dronacharya,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,None,None
...,...,...,...,...,...,...,...,...,...,...,...
3196,node,9565595809,POINT (77.31645 28.34000),station,None,Raja Nahar Singh,Delhi Metro,Delhi Metro Rail Corporation Limited,station,None,None
3197,node,9597709830,POINT (77.26428 28.54455),station,None,Govind Puri,Delhi Metro,Delhi Metro Rail Corporation Limited,station,None,None
3198,node,9597709831,POINT (77.27535 28.54318),station,None,Harkesh Nagar Okhla,Delhi Metro,Delhi Metro Rail Corporation Limited,station,OKA,None
3199,node,9937948255,POINT (77.26484 28.55445),stop,None,Okhla NSIC,Delhi Metro,Delhi Metro Rail Corporation Limited,stop_position,None,None


In [39]:
%%time

query = f"""
SELECT COUNT(*)
FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet' AS b, ST_Read({aoi_file}) as aoi
WHERE ST_Intersection(ST_GeomFromWKB(b.geometry), ST_GeomFromWKB(aoi.geom))
"""

x = duckdb.sql(query)

CPU times: user 76.6 ms, sys: 13.2 ms, total: 89.8 ms
Wall time: 751 ms


In [ ]:
%%time
stations_data_url = "'stations_stops.parquet'"
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country"
country_iso = "IND"
duckdb.sql(f"CREATE TABLE lso_buildings AS SELECT * FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet' AS buildings")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [42]:
duckdb.sql("SELECT COUNT(*) FROM lso_buildings_google").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1394225 │
└──────────────┘



In [ ]:
# %%time
# stations_data_url = "'stations_stops.parquet'"
# prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
# partitions = "by_country"
# country_iso = "IND"

# duckdb.sql(f"CREATE TABLE ind_buildings_google AS SELECT * FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# %%time
# stations_data_url = "'stations_stops.parquet'"
# prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
# partitions = "by_country"
# country_iso = "IND"

# bdf = duckdb.sql(f'''
#     LOAD spatial;
#     LOAD httpfs;
#     COPY(
#         SELECT * FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet' AS buildings
#         JOIN read_parquet({stations_data_url}) as stations
#         ON ST_Intersects(ST_GeomFromWKB(buildings.geometry), ST_GeomFromWKB(stations.geometry))
#     ) TO 'microsoft_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
# ''')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country"
country_iso = "IND"

# Use single file
duckdb.sql(f"SELECT count(*) FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet'").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    520127455 │
└──────────────┘

